In [1]:
pip install gTTS

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -U gradio

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

import gradio as gr

from gtts import gTTS
from IPython.display import Audio
import os
from PyPDF2 import PdfReader 
# 載入模型和標記器
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")

# 定義一個翻譯函數
def translate(text, target_lang_code):
    # 將輸入文本進行標記化
    model_inputs = tokenizer(text, return_tensors="pt")
    
    # 生成翻譯
    generated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id[target_lang_code]
    )
    
    # 解碼翻譯後的標記
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    return translated_text[0]

# 從使用者輸入文本
article_en = input("請輸入需要翻譯的文本: ")

# 翻譯成印地語
translation_hi = translate(article_en, "hi_IN")
print("印地語翻譯:", translation_hi)

# 翻譯成中文
translation_zh = translate(article_en, "zh_CN")
print("中文翻譯:", translation_zh)

# 翻譯成韓語
translation_ko = translate(article_en, "ko_KR")
print("韓語翻譯:", translation_ko)

# 翻譯成日語
translation_ja = translate(article_en, "ja_XX")
print("日語翻譯:", translation_ja)

請輸入需要翻譯的文本:  晚安


印地語翻譯: रात्रिकाल
中文翻譯: 晚安
韓語翻譯: 晚安
日語翻譯: 晚安


In [11]:
import gradio as gr
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

import tempfile

from gtts import gTTS
from IPython.display import Audio
import os
from PyPDF2 import PdfReader 

# 載入模型和標記器
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")

# 定義後處理函數，去除半語結尾
def remove_semilang(sentence):
    if sentence.endswith("하다"):
        return sentence[:-2] + "해"
    elif sentence.endswith("다"):
        return sentence[:-1]
    elif sentence.endswith("합니다"):
        return sentence[:-3]
    elif sentence.endswith("세요"):
        return sentence[:-2]
    elif sentence.endswith("하기"):
        return sentence[:-1] + "해"
    elif sentence.endswith("기"):
        return sentence[:-1]
    else:
        return sentence

# 定義翻譯函數
def translate_text(text, wordtype):
    target_lang_code = "ko_KR"  # 預設翻譯成韓語
    model_inputs = tokenizer(text, return_tensors="pt")
    generated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id[target_lang_code]
    )
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    return translated_text[0]

# 定義半語翻譯函數
def translate_text_half(text, wordtype):
    target_lang_code = "ko_KR"  # 預設翻譯成韓語
    model_inputs = tokenizer(text, return_tensors="pt")
    generated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id[target_lang_code]
    )
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    semi_lang_translated = remove_semilang(translated_text[0])
    return semi_lang_translated

# 定義其他語言翻譯函數
def translate_text2(text):
    target_lang_code = "ja_XX"  # 翻譯成日語
    model_inputs = tokenizer(text, return_tensors="pt")
    generated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id[target_lang_code]
    )
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    return translated_text[0]

def translate_text3(text):
    target_lang_code = "zh_CN"  # 翻譯成中文
    model_inputs = tokenizer(text, return_tensors="pt")
    generated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id[target_lang_code]
    )
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    return translated_text[0]

def text_to_speech(text):
    language = "ko"
    # 使用tempfile生成臨時文件名
    with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as temp_file:
        gtts_object = gTTS(text=text, lang=language, slow=False)
        gtts_object.save(temp_file.name)
        return temp_file.name

def extract_and_translate_text_from_pdf(pdf_file, max_length=200):
    try:
        text = ""
        with open(pdf_file.name, "rb") as file:
            reader = PdfReader(file)
            for page in reader.pages:
                text += page.extract_text() or ""
        
        # 分割文本以确保不超过模型的最大输入长度
        chunks = [text[i:i+max_length] for i in range(0, len(text), max_length)]
        
        # 翻譯提取的文本
        translated_text = ""
        for chunk in chunks:
            model_inputs = tokenizer(chunk, return_tensors="pt")
            generated_tokens = model.generate(
                **model_inputs,
                forced_bos_token_id=tokenizer.lang_code_to_id["zh_CN"]
            )
            translated_chunk = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
            translated_text += translated_chunk + " "  # 在每段翻译结果之间加上空格
        
        return translated_text
    except Exception as e:
        return str(e)

# 定義Gradio介面
with gr.Blocks() as demo:
    gr.Markdown("翻譯成韓文，不然我真的會gg耶，拜託拜託，阿如果有瑕疵的不要噴太用力嗚嗚嗚嗚嗚嗚嗚")

    with gr.Tab("一般"):
        with gr.Column():
            text_input = gr.Textbox(label="請輸入文本")
            wordtype_input = gr.Radio(choices=["動詞", "名詞", "形容詞"], label="請選擇詞類")
            text_output = gr.Textbox(label="翻譯結果")
            text_button = gr.Button("翻譯")

    with gr.Tab("半語"):
        with gr.Column():
            text_input1 = gr.Textbox(label="請輸入文本")
            wordtype_input1 = gr.Radio(choices=["動詞", "名詞", "形容詞"], label="請選擇詞類")
            text_output1 = gr.Textbox(label="翻譯結果")
            text_button1 = gr.Button("翻譯")

    with gr.Tab("音檔"):
        with gr.Column():
            text_input2 = gr.Textbox(label="請輸入文本")
            text_output2 = gr.Audio(label="音檔")
            text_button2 = gr.Button("轉換")

    with gr.Tab("日文"):
        with gr.Column():
            text_input3 = gr.Textbox(label="請輸入文本")
            text_output3 = gr.Textbox(label="翻譯結果")
            text_button3 = gr.Button("翻譯")

    with gr.Tab("中文"):
        with gr.Column():
            text_input4 = gr.Textbox(label="請輸入文本")
            text_output4 = gr.Textbox(label="翻譯結果")
            text_button4 = gr.Button("翻譯")

    with gr.Tab("pdf的翻譯"):
        with gr.Column():
            pdf_input = gr.File(label="上傳 PDF", type="filepath")
            pdf_output = gr.Textbox(placeholder="翻譯後的文字將顯示在這裡", label="翻譯")
            pdf_button = gr.Button("翻譯")

    # 定義翻譯或替換函數
    def translate_or_replace(text):
        if text in ["早安", "晚安", "午安"]:
            return "안녕하세요"
        else:
            return translate_text(text, "ko_KR")

    # 定義按鈕的事件
    text_button.click(translate_or_replace, inputs=[text_input], outputs=text_output)
    text_button1.click(translate_or_replace, inputs=[text_input1], outputs=text_output1)
    text_button2.click(text_to_speech, inputs=text_input2, outputs=text_output2)
    text_button3.click(lambda text: translate_text2(text), inputs=text_input3, outputs=text_output3)
    text_button4.click(lambda text: translate_text3(text), inputs=text_input4, outputs=text_output4)
    pdf_button.click(extract_and_translate_text_from_pdf, inputs=pdf_input, outputs=pdf_output)

# 啟動Gradio介面
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7864
Running on public URL: https://b2337f22c4376d848f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
